# Downloading the data

In [ ]:
import requests
import json

url = "https://samate.nist.gov/SARD/api/test-cases/search?page={}&limit=100"
output_file = "results2.json"

def fetch_page(page_number):
    response = requests.get(url.format(page_number))
    data = response.json()
    return data

def main():
    response = fetch_page(1)
    total_pages = response['pageCount']

    with open(output_file, 'w') as f:
        json.dump(response, f)
        f.write('\n')

        for page_number in range(2, total_pages + 1):
            data = fetch_page(page_number)
            json.dump(data, f)
            f.write('\n')

    print(f"Retrieved {total_pages} results and saved them to {output_file}")

if __name__ == "__main__":
    main()


In [9]:
import pandas as pd
import json

input_file = "results2.json"

# Load the JSON data line by line
test_cases = []
with open(input_file) as f:
    for line in f:
        test_cases.append(json.loads(line.strip()))

# Load the test cases into a DataFrame
df = pd.DataFrame(test_cases)


In [12]:
# Use list comprehension and pd.concat to split the arrays of dictionaries into individual rows
expanded_df = pd.concat([pd.DataFrame(df_testCases.iloc[i]) for i in range(len(df_testCases))], ignore_index=True)

# Display the expanded DataFrame
expanded_df.head()

,download,link,identifier,sarif
0,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/248681...,248681-v2.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...
1,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/248604...,248604-v2.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...
2,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/248647...,248647-v2.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...
3,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/248686...,248686-v2.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...
4,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/248575...,248575-v2.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...


In [41]:
import pandas as pd

# Assuming new_expanded_df is your DataFrame
non_dict_values = new_expanded_df['taxonomies'].apply(lambda x: x if not isinstance(x, dict) else None)
non_dict_values = non_dict_values.dropna()

if not non_dict_values.empty:
    print("Non-dictionary values found in the 'taxonomies' column:")
    print(non_dict_values)
else:
    print("All values in the 'taxonomies' column are dictionaries.")


All values in the 'taxonomies' column are dictionaries.


In [13]:
def process_sarif(sarif):
    runs = sarif['runs']
    result = []

    for run in runs:
        if 'taxonomies' in run:
            for taxonomy in run['taxonomies']:
                if 'taxa' in taxonomy:
                    for taxa in taxonomy['taxa']:
                        id = taxa.get('id', None)
                        name = taxa.get('name', None)
                        downloadUri = taxonomy.get('downloadUri', None)
                        result.append({'id': id, 'name': name, 'downloadUri': downloadUri})

    return result

# Apply the process_sarif function to the 'sarif' column
expanded_df['taxonomies'] = expanded_df['sarif'].apply(process_sarif)

# Create a new dataframe by exploding the 'taxonomies' column
new_expanded_df = expanded_df.explode('taxonomies')

# Split the 'taxonomies' column into separate columns for 'id', 'name', and 'downloadUri'
new_expanded_df[['id', 'name', 'downloadUri']] = pd.DataFrame(new_expanded_df['taxonomies'].tolist(), index=new_expanded_df.index)

# Drop the 'taxonomies' column
new_expanded_df = new_expanded_df.drop(columns=['taxonomies'])

# Display the new dataframe
print(new_expanded_df)


AttributeError: 'float' object has no attribute 'keys'

In [58]:
import pandas as pd

def process_artifacts(sarif):
    runs = sarif['runs']
    result = []

    for run in runs:
        if 'artifacts' in run:
            for artifact in run['artifacts']:
                length = artifact.get('length', None)
                uri = artifact['location'].get('uri', None) if 'location' in artifact else None
                sourceLanguage = artifact.get('sourceLanguage', None)
                result.append({'length': length, 'uri': uri, 'sourceLanguage': sourceLanguage})

    return result

# Apply the process_artifacts function to the 'sarif' column
expanded_df['artifacts'] = expanded_df['sarif'].apply(process_artifacts)

# Create a new dataframe by exploding the 'artifacts' column
new_artifacts_df = expanded_df.explode('artifacts')

# Split the 'artifacts' column into separate columns for 'length', 'uri', and 'sourceLanguage'
new_artifacts_df[['length', 'uri', 'sourceLanguage']] = pd.DataFrame(new_artifacts_df['artifacts'].tolist(), index=new_artifacts_df.index)

# Drop the 'artifacts' column
new_artifacts_df = new_artifacts_df.drop(columns=['artifacts'])

# Display the new dataframe
print(new_artifacts_df)


                                                 download  \
0       https://samate.nist.gov/SARD/downloads/version...   
0       https://samate.nist.gov/SARD/downloads/version...   
0       https://samate.nist.gov/SARD/downloads/version...   
0       https://samate.nist.gov/SARD/downloads/version...   
1       https://samate.nist.gov/SARD/downloads/version...   
...                                                   ...   
103995  https://samate.nist.gov/SARD/downloads/version...   
103996  https://samate.nist.gov/SARD/downloads/version...   
103997  https://samate.nist.gov/SARD/downloads/version...   
103998  https://samate.nist.gov/SARD/downloads/version...   
103999  https://samate.nist.gov/SARD/downloads/version...   

                                                     link     identifier  \
0       https://samate.nist.gov/SARD/test-cases/248681...  248681-v2.0.0   
0       https://samate.nist.gov/SARD/test-cases/248681...  248681-v2.0.0   
0       https://samate.nist.gov/SARD/te

In [59]:
import pandas as pd

def process_results(sarif):
    runs = sarif['runs']
    result = []

    for run in runs:
        if 'results' in run:
            for res in run['results']:
                ruleId = res.get('ruleId', None)
                text = res['message'].get('text', None) if 'message' in res else None
                if 'locations' in res and len(res['locations']) == 1:
                    location = res['locations'][0]
                    if 'physicalLocation' in location:
                        physical_location = location['physicalLocation']
                        startLine = physical_location['region'].get('startLine', None) if 'region' in physical_location else None
                        uri = physical_location['artifactLocation'].get('uri', None) if 'artifactLocation' in physical_location else None
                        result.append({'ruleId': ruleId, 'text': text, 'startLine': startLine, 'uri': uri})

    return result

# Apply the process_results function to the 'sarif' column
new_artifacts_df['results'] = new_artifacts_df['sarif'].apply(process_results)

# Create a new dataframe by exploding the 'results' column
new_results_df = new_artifacts_df.explode('results')

# Split the 'results' column into separate columns for 'ruleId', 'text', 'startLine', and 'uri'
new_results_df[['ruleId', 'text', 'startLine', 'uri']] = pd.DataFrame(new_results_df['results'].tolist(), index=new_results_df.index)

# Drop the 'results' column
new_results_df = new_results_df.drop(columns=['results'])

# Display the new dataframe
print(new_results_df)


AttributeError: 'float' object has no attribute 'keys'

In [60]:
import pandas as pd

def process_results(sarif):
    runs = sarif['runs']
    result = []

    for run in runs:
        if 'results' in run:
            for res in run['results']:
                ruleId = res.get('ruleId', None)
                text = res['message'].get('text', None) if 'message' in res else None
                if 'locations' in res and len(res['locations']) == 1:
                    location = res['locations'][0]
                    if 'physicalLocation' in location:
                        physical_location = location['physicalLocation']
                        startLine = physical_location['region'].get('startLine', None) if 'region' in physical_location else None
                        uri = physical_location['artifactLocation'].get('uri', None) if 'artifactLocation' in physical_location else None
                        result.append({'ruleId': ruleId, 'text': text, 'startLine': startLine, 'uri': uri})

    return result

# Apply the process_results function to the 'sarif' column
new_artifacts_df['results'] = new_artifacts_df['sarif'].apply(process_results)

# Create a new dataframe by exploding the 'results' column
new_results_df = new_artifacts_df.explode('results')

# Split the 'results' column into separate columns for 'ruleId', 'text', 'startLine', and 'uri'
new_results_df[['ruleId', 'text', 'startLine', 'uri']] = pd.DataFrame(new_results_df['results'].tolist(), index=new_results_df.index)

# Drop the 'results' column
new_results_df = new_results_df.drop(columns=['results'])

# Display the new dataframe
print(new_results_df)


AttributeError: 'float' object has no attribute 'keys'

In [61]:
import pandas as pd

def process_properties(sarif):
    runs = sarif['runs']
    result = []

    for run in runs:
        properties = run['properties']

        # Extract the additional fields from the properties dictionary
        ID = properties.get('id', None)
        language = properties.get('language', None)
        description = properties.get('description', None)
        operating_system = properties.get('operatingSystem', None)
        submission_date = properties.get('submissionDate', None)
        author = properties.get('author', None)
        state = properties.get('state', None)
        type_ = properties.get('type', None)

        result.append({
            'ID': ID,
            'language': language,
            'description': description,
            'operating_system': operating_system,
            'submission_date': submission_date,
            'author': author,
            'state': state,
            'type': type_
        })

    return result

# Apply the process_properties function to the 'sarif' column
new_results_df['properties'] = new_results_df['sarif'].apply(process_properties)

# Create a new dataframe by exploding the 'properties' column
new_properties_df = new_results_df.explode('properties')

# Split the 'properties' column into separate columns for the extracted fields
new_properties_df[['ID', 'language', 'description', 'operating_system', 'submission_date', 'author', 'state', 'type']] = pd.DataFrame(new_properties_df['properties'].tolist(), index=new_properties_df.index)

# Drop the 'properties' column
new_properties_df = new_properties_df.drop(columns=['properties'])

# Display the new dataframe
new_properties_df


,download,link,identifier,sarif,taxonomies,length,uri,sourceLanguage,results,ID,language,description,operating_system,submission_date,author,state,type
0,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/248681...,248681-v2.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...,"[{'id': '78', 'name': 'Improper Neutralization...",7719,src/testcases/CWE78_OS_Command_Injection/s09/C...,c,"{'ruleId': 'CWE-78', 'text': 'Improper Neutral...",248681,c,CWE: 78 OS Command Injection\n BadSource: list...,cpe:2.3:o:microsoft:windows:*:*:*:*:*:*:*:*,2017-10-06,NSA/Center for Assured Software,mixed,source code
0,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/248681...,248681-v2.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...,"[{'id': '78', 'name': 'Improper Neutralization...",5429,src/testcasesupport/io.c,c,"{'ruleId': 'CWE-78', 'text': 'Improper Neutral...",248681,c,CWE: 78 OS Command Injection\n BadSource: list...,cpe:2.3:o:microsoft:windows:*:*:*:*:*:*:*:*,2017-10-06,NSA/Center for Assured Software,mixed,source code
0,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/248681...,248681-v2.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...,"[{'id': '78', 'name': 'Improper Neutralization...",4004,src/testcasesupport/std_testcase.h,c,"{'ruleId': 'CWE-78', 'text': 'Improper Neutral...",248681,c,CWE: 78 OS Command Injection\n BadSource: list...,cpe:2.3:o:microsoft:windows:*:*:*:*:*:*:*:*,2017-10-06,NSA/Center for Assured Software,mixed,source code
0,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/248681...,248681-v2.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...,"[{'id': '78', 'name': 'Improper Neutralization...",1457,src/testcasesupport/std_testcase_io.h,c,"{'ruleId': 'CWE-78', 'text': 'Improper Neutral...",248681,c,CWE: 78 OS Command Injection\n BadSource: list...,cpe:2.3:o:microsoft:windows:*:*:*:*:*:*:*:*,2017-10-06,NSA/Center for Assured Software,mixed,source code
1,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/248604...,248604-v2.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...,"[{'id': '78', 'name': 'Improper Neutralization...",6249,src/testcases/CWE78_OS_Command_Injection/s09/C...,c,"{'ruleId': 'CWE-78', 'text': 'Improper Neutral...",248604,c,CWE: 78 OS Command Injection\n BadSource: list...,cpe:2.3:o:microsoft:windows:*:*:*:*:*:*:*:*,2017-10-06,NSA/Center for Assured Software,mixed,source code
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103995,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/183/ve...,183-v1.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...,"[{'id': '119', 'name': 'Improper Restriction o...",1947,src/basic-00018-large.c,c,"{'ruleId': 'CWE-119', 'text': 'Improper Restri...",183,c,Buffer Overflow. This code has been donated by...,None,2005-12-27,MIT,bad,source code
103996,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/170/ve...,170-v1.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...,"[{'id': '119', 'name': 'Improper Restriction o...",1964,src/basic-00014-ok.c,c,"{'ruleId': 'CWE-119', 'text': 'Improper Restri...",170,c,Buffer Overflow. This code has been donated by...,None,2005-12-27,MIT,good,source code
103997,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/171/ve...,171-v1.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...,"[{'id': '119', 'name': 'Improper Restriction o...",1902,src/basic-00015-large.c,c,"{'ruleId': 'CWE-119', 'text': 'Improper Restri...",171,c,Buffer Overflow. This code has been donated by...,None,2005-12-27,MIT,bad,source code
103998,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/116/ve...,116-v1.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...,"[{'id': '119', 'name'

In [62]:
parsed_dataset = new_properties_df[new_properties_df['language'] == 'java']

In [64]:
parsed_dataset.to_csv('parse_data_try1.csv')

In [65]:
non_empty_taxonomies = parsed_dataset[parsed_dataset['taxonomies'].notna()]


In [169]:
non_empty_taxonomies = valid_exploded_taxonomies[valid_exploded_taxonomies['results'].notna()]


In [66]:
non_empty_taxonomies.to_csv('SARD_Dataset.csv')

In [70]:
import os
import requests
import zipfile

# Get unique download URLs
unique_download_urls = non_empty_taxonomies['download'].unique()

# Create a directory for zip files if it doesn't exist
download_directory = "downloaded_zip_files"
if not os.path.exists(download_directory):
    os.makedirs(download_directory)

# Download and extract zip files
for url in unique_download_urls:
    response = requests.get(url)
    zip_file_name = os.path.join(download_directory, os.path.basename(url))
    
    # Save the downloaded zip file
    with open(zip_file_name, 'wb') as zip_file:
        zip_file.write(response.content)
    


In [71]:
import os
import zipfile

root_directory = r"/Users/connor.neff/Desktop/Education/CSCE5290-NLPProject"
download_directory = os.path.join(root_directory, "downloaded_zip_files")
output_directory = os.path.join(root_directory, "java_source_code_files")

def unzip_files(download_directory, output_directory):
    # Check if output_directory exists, if not, create it
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Iterate through all the files in download_directory
    for filename in os.listdir(download_directory):
        # Check if the file is a zip file
        if filename.endswith(".zip"):
            # Construct the full path of the zip file
            zip_path = os.path.join(download_directory, filename)

            # Create a folder for the current zip file using its name (without the extension)
            current_output_directory = os.path.join(output_directory, os.path.splitext(filename)[0])
            if not os.path.exists(current_output_directory):
                os.makedirs(current_output_directory)

            # Open the zip file
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                # Extract the contents of the zip file to the current_output_directory
                zip_ref.extractall(current_output_directory)

            print(f"Extracted {filename} to {current_output_directory}")

if __name__ == "__main__":
    unzip_files(download_directory, output_directory)


Extracted 55731-v1.0.0.zip to /Users/connor.neff/Desktop/Education/CSCE5290-NLPProject/java_source_code_files/55731-v1.0.0
Extracted 127390-v1.0.0.zip to /Users/connor.neff/Desktop/Education/CSCE5290-NLPProject/java_source_code_files/127390-v1.0.0
Extracted 59079-v1.0.0.zip to /Users/connor.neff/Desktop/Education/CSCE5290-NLPProject/java_source_code_files/59079-v1.0.0
Extracted 125055-v1.0.0.zip to /Users/connor.neff/Desktop/Education/CSCE5290-NLPProject/java_source_code_files/125055-v1.0.0
Extracted 50231-v1.0.0.zip to /Users/connor.neff/Desktop/Education/CSCE5290-NLPProject/java_source_code_files/50231-v1.0.0
Extracted 124249-v1.0.0.zip to /Users/connor.neff/Desktop/Education/CSCE5290-NLPProject/java_source_code_files/124249-v1.0.0
Extracted 59566-v1.0.0.zip to /Users/connor.neff/Desktop/Education/CSCE5290-NLPProject/java_source_code_files/59566-v1.0.0
Extracted 48283-v1.0.0.zip to /Users/connor.neff/Desktop/Education/CSCE5290-NLPProject/java_source_code_files/48283-v1.0.0
Extracted 

In [3]:
import os
import zipfile
import pandas as pd

root_directory = r"/Users/connor.neff/Desktop/Education/CSCE5290-NLPProject"
csv_file_path = os.path.join(root_directory, "SARD_Dataset.csv")
source_code_directory = os.path.join(root_directory, "java_source_code_files")

# Read the CSV file using pandas
data = pd.read_csv(csv_file_path)
count = 0
# Create a dictionary to store the source code files associated with each identifier
identifier_to_source_code = {}



/var/folders/lw/n2j6t0310r7gsdn2dl3544p80000gp/T/ipykernel_17183/3477692290.py:10: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(csv_file_path)


In [74]:
# functional code use this
import os
import pandas as pd

def find_java_files(path):
    java_files = []
    for root, _, files in os.walk(path):
        for file in files:
            if file.endswith('.java'):
                java_files.append(os.path.join(root, file))
    return java_files

root_directory = r"/Users/connor.neff/Desktop/Education/CSCE5290-NLPProject"
csv_file_path = os.path.join(root_directory, "SARD_Dataset.csv")
source_code_directory = os.path.join(root_directory, "java_source_code_files")

# Read the CSV file using pandas
data = non_empty_taxonomies
count = 0
# Create a list to store DataFrames for each identifier
source_code_dataframes = []

for identifier in data['identifier']:
    # Convert the identifier to a string and check if a folder with that name exists in the source_code_directory
    identifier_folder = os.path.join(source_code_directory, identifier)  # Convert '-' to '.' in the identifier
    if os.path.exists(identifier_folder):
        # Get all the .java files in the identifier_folder
        java_files = find_java_files(identifier_folder)

        # Read the contents of each .java file and store them in a list
        source_code_list = []
        for java_file in java_files:
            with open(java_file, 'r', encoding='utf-8', errors='replace') as file:
                source_code_list.append(file.read())

        # Create a DataFrame with the source code list and the identifier
        source_code_df = pd.DataFrame({'identifier': [identifier] * len(source_code_list),
                                       'source_code': source_code_list})

        # Append the DataFrame to the list of DataFrames
        source_code_dataframes.append(source_code_df)
    else:
        print(f"No folder found for identifier {identifier}")
        count = count + 1

# Concatenate all the DataFrames into one DataFrame
combined_source_code_data = pd.concat(source_code_dataframes, ignore_index=True)

# Merge the original data DataFrame with the combined_source_code_data DataFrame
combined_data = data.merge(combined_source_code_data, on='identifier')

# Now you can access the source code files for each identifier in the combined_data DataFrame
print(count)


0


In [77]:
combined_data.to_csv('/Users/connor.neff/Desktop/Education/CSCE5290-NLPProject/combined_data.csv')

# Starting the parsing of the data

In [130]:
import antlr4
from JavaLexer import JavaLexer
from JavaParser import JavaParser
from JavaParserListener import JavaParserListener
from antlr4 import ParseTreeWalker

def parse_code(row, col_name):
    code = row[col_name]
    input_stream = antlr4.InputStream(code)
    lexer = JavaLexer(input_stream)
    token_stream = antlr4.CommonTokenStream(lexer)
    parser = JavaParser(token_stream)
    tokens = lexer.getAllTokens()
    tokenArr = []
    for i in tokens:
        tokenArr.append(i.text)
    
    return tokenArr

combined_data2['tokens_list'] = combined_data2.apply(parse_code, col_name='source_code', axis=1)


line 3:20 token recognition error at: '".getBytes();\n'


In [132]:
combined_data2.to_csv('parsedDataWLexer2.csv')

In [134]:
combined_data2.head()

,download,link,identifier,sarif,taxonomies,length,uri,sourceLanguage,results,ID,language,description,operating_system,submission_date,author,state,type,source_code,tokens_list
0,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/129938...,129938-v1.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...,"[{'id': '190', 'name': 'Integer Overflow or Wr...",17344,src/main/java/testcases/CWE190_Integer_Overflo...,java,NaN,129938,java,CWE: 190 Integer Overflow\n BadSource: listen_...,None,2013-05-21,None,mixed,source code,/*\n* @description Helper IO class\n* \n*\n* *...,[/*\n* @description Helper IO class\n* \n*\n* ...
1,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/129938...,129938-v1.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...,"[{'id': '190', 'name': 'Integer Overflow or Wr...",17344,src/main/java/testcases/CWE190_Integer_Overflo...,java,NaN,129938,java,CWE: 190 Integer Overflow\n BadSource: listen_...,None,2013-05-21,None,mixed,source code,/*\n@description This abstract class is the ba...,[/*\n@description This abstract class is the b...
2,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/129938...,129938-v1.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...,"[{'id': '190', 'name': 'Integer Overflow or Wr...",17344,src/main/java/testcases/CWE190_Integer_Overflo...,java,NaN,129938,java,CWE: 190 Integer Overflow\n BadSource: listen_...,None,2013-05-21,None,mixed,source code,/*\n@description This abstract class is the ba...,[/*\n@description This abstract class is the b...
3,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/129938...,129938-v1.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...,"[{'id': '190', 'name': 'Integer Overflow or Wr...",17344,src/main/java/testcases/CWE190_Integer_Overflo...,java,NaN,129938,java,CWE: 190 Integer Overflow\n BadSource: listen_...,None,2013-05-21,None,mixed,source code,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,[/* TEMPLATE GENERATED TESTCASE FILE\nFilename...
4,https://samate.nist.gov/SARD/downloads/version...,https://samate.nist.gov/SARD/test-cases/129938...,129938-v1.0.0,{'runs': [{'tool': {'driver': {'name': 'SARD -...,"[{'id': '190', 'name': 'Integer Overflow or Wr...",17344,src/main/java/testcases/CWE190_Integer_Overflo...,java,NaN,129938,java,CWE: 190 Integer Overflow\n BadSource: listen_...,None,2013-05-21,None,mixed,source code,/*\n* @description Helper IO class\n* \n*\n* *...,[/*\n* @description Helper IO class\n* \n*\n* ...


In [137]:
combined_data3 = combined_data2[['source_code']]

In [138]:
combined_data3

,source_code
0,/*\n* @description Helper IO class\n* \n*\n* *...
1,/*\n@description This abstract class is the ba...
2,/*\n@description This abstract class is the ba...
3,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...
4,/*\n* @description Helper IO class\n* \n*\n* *...
...,...
1265079,import java.io.*;\n\npublic class PathManipula...
1265080,/*\nDescription: Tainted input allows arbitrar...
1265081,/*\nDescription: Tainted input allows arbitrar...
1265082,{\nint month = 8;\nswitch (month) {\ncase 1: p...


In [ ]:
from antlr4 import *
from JavaLexer import JavaLexer
from JavaParser import JavaParser
from JavaParserListener import JavaParserListener

class JavaContextListener(JavaParserListener):
    def __init__(self):
        self.context_dict = {}

    def enterEveryRule(self, ctx):
        context_name = ctx.__class__.__name__
        node_text = ctx.getText()
        
        if context_name not in self.context_dict:
            self.context_dict[context_name] = []
        self.context_dict[context_name].append(node_text)

    def get_context_dict(self):
        return self.context_dict

def parse_and_create_context_dict(row, col_name):
    java_code = row[col_name]
    input_stream = InputStream(java_code)
    lexer = JavaLexer(input_stream)
    token_stream = CommonTokenStream(lexer)
    parser = JavaParser(token_stream)
    tree = parser.compilationUnit()
    
    context_listener = JavaContextListener()
    walker = ParseTreeWalker()
    walker.walk(context_listener, tree)
    
    return context_listener.get_context_dict()


combined_data3['ASTFlat'] = combined_data3.apply(parse_and_create_context_dict, col_name='source_code', axis=1)


print(combined_data3['ASTFlat'])


# CWE Information

In [ ]:
from cwe import Database

def getCWEData(row,col_name):
    db = Database()
    cweid = row[col_name]
    id = cweid.split('-')
    if (len(id) > 1):
        weakness = db.get(id[1])
        return weakness.to_dict()
    return ""


combined_data3['cweData'] = combined_data3.apply(getCWEData, col_name='ruleId', axis=1)
combined_data3.head()

# Scratch

In [128]:
!java -jar ~/Downloads/antlr-4.9.2-complete.jar -Dlanguage=Python3 JavaParser.g4


In [129]:
! pip install antlr4-python3-runtime==4.9.2 --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 971.5 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.2-py3-none-any.whl size=144567 sha256=d8453412cd0aac85896af91299b01b089b180eb17c93368a98804e7c4b0d2c25
  Stored in directory: /Users/connor.neff/Library/Caches/pip/wheels/0c/dc/1c/57974cae05c0a98c8d8590c471bdd8e953fcf18b14fa0890b7
Successfully built antlr4-python3-runtime
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.12.0
    Uninstalling antlr4-python3-runtime-4.12.0:
      Successfully uninstalled antlr4-python3-runtime-4.12.0


In [14]:
test = data['identifier'].iloc[0]

identifier_folder = os.path.join(source_code_directory, test) 
identifier_folder

'/Users/connor.neff/Desktop/Education/CSCE5290-NLPProject/java_source_code_files/129923-v1.0.0'

In [15]:
os.path.exists(identifier_folder)

True

In [18]:
def find_java_files(path):
    java_files = []
    for root, _, files in os.walk(path):
        for file in files:
            if file.endswith('.java'):
                java_files.append(os.path.join(root, file))
    return java_files

In [19]:
java_files = find_java_files(identifier_folder)

In [226]:
combined_data3 = combined_data2

In [228]:
combined_data3.to_csv('dataset_full040423.csv')

In [229]:
combined_data4 = combined_data3

In [116]:
combined_data3.to_csv('parsedDataWParser.csv')

Error: No such keg: /usr/local/Cellar/antlr


In [114]:
!antlr4 -Dlanguage=Python3 JavaParser.g4


# Model

In [ ]:
#Next Feature